# What's cooking?

bert

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
train_df = pd.read_json('./train.json')
test_df = pd.read_json('./test.json')

train_df.sample(5)

,id,cuisine,ingredients
26197,21245,chinese,"[soy sauce, sesame oil, garlic cloves, boneles..."
16894,9769,thai,"[chili pepper, white cabbage, garlic, fresh li..."
12457,20301,italian,"[large eggs, salt, chopped fresh chives, fresh..."
9556,33891,mexican,"[sugar, whipping cream, eggs, semisweet chocol..."
24621,16747,italian,"[mozzarella cheese, toasted pine nuts, grated ..."


In [14]:
train = train_df.copy()
train['X'] = train_df.ingredients.str.join(' ')

train.head()

,id,cuisine,ingredients,X
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...",romaine lettuce black olives grape tomatoes ga...
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",plain flour ground pepper salt tomatoes ground...
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",eggs pepper salt mayonaise cooking oil green c...
3,22213,indian,"[water, vegetable oil, wheat, salt]",water vegetable oil wheat salt
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...",black pepper shallots cornflour cayenne pepper...


In [15]:
# encode (label) targets:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
cuisine = label_encoder.fit_transform(train_df['cuisine'].values)
cuisine

array([ 6, 16,  4, ...,  8,  3, 13])

In [16]:
# join data
y = cuisine.copy()
X = train.X.copy()
X.shape

(39774,)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(X_train.shape)
print(y_train.shape)

(26648,)
(26648,)


In [18]:
# Loading Dependencies
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers

from tokenizers import BertWordPieceTokenizer

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Num GPUs Available:  1
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5



In [23]:
from tqdm.notebook import tqdm
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 14.5kB/s]
Downloading: 100%|██████████| 570/570 [00:00<00:00, 569kB/s]
Downloading: 100%|██████████| 208k/208k [00:00<00:00, 288kB/s]  
Downloading: 100%|██████████| 426k/426k [00:01<00:00, 343kB/s]  


In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, random_state=42)
print(X_train.shape)
print(y_train.shape)

(33807,)
(33807,)


In [41]:
%%time

x_train = tokenizer(X_train.values.tolist(), padding=True, truncation=True, return_tensors="tf")
x_valid = tokenizer(X_valid.values.tolist(), padding=True, truncation=True, return_tensors="tf")
x_test = tokenizer(X_test.values.tolist(), padding=True, truncation=True, return_tensors="tf")

Wall time: 7.59 s


In [45]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * 5
MAX_LEN = 192

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

def build_model(transformer, max_len=512):
    """
    function for training the BERT model
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [46]:
from transformers import TFDistilBertModel
transformer_layer = (
    transformers.TFDistilBertModel
    .from_pretrained('distilbert-base-multilingual-cased')
)
model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Downloading:  60%|██████    | 522M/869M [1:14:39<49:28, 122kB/s]    


OSError: [WinError 127] The specified procedure could not be found. Error loading "c:\Users\alexa\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\lib\cublas64_11.dll" or one of its dependencies.

In [ ]:
%%time
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)


In [ ]:
# xtest_seq = token.texts_to_sequences(X_test)
# xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)
# test_results = model.evaluate(x=xtest_pad,y= y_test, verbose=1)
# print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')